In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import math
import re
import ast

In [2]:
aero = pd.read_csv('aeronefs.csv')
compo = pd.read_csv('composants.csv')
deg = pd.read_csv('degradations_2024-02-05_FULL.csv')
vol = pd.read_csv('logs_vols_2024-02-05_FULL.csv')

In [3]:
# passage au type datetime des df
aero['debut_service'] = pd.to_datetime(aero['debut_service'])
aero['last_maint'] = pd.to_datetime(aero['last_maint'])
aero['end_maint'] = pd.to_datetime(aero['end_maint'],errors='coerce')
aero['end_maint'] = aero['end_maint'].dt.strftime('%Y-%m-%d')
aero['end_maint'] = pd.to_datetime(aero['end_maint'],errors='coerce')
deg['measure_day'] = pd.to_datetime(deg['measure_day'])
vol['jour_vol'] = pd.to_datetime(vol['jour_vol'])

# modification des noms de colonne servant de clés, la base gardé étant le nom du csv 'aeronefs' à savoir 'ref_aero'
colonne_ref = 'ref_aero'
vol = vol.rename(columns={'aero_linked':colonne_ref})
deg = deg.rename(columns={'linked_aero':colonne_ref})
compo = compo.rename(columns={'aero':colonne_ref})

# modif noms de colonne des composants
deg = deg.rename(columns={'compo_concerned':'ref_compo'})

# application d'un arrondi sur les floats 
compo['taux_usure_actuel'] = compo['taux_usure_actuel'].apply(lambda x: math.ceil(x * 100) / 100)
deg['usure_nouvelle'] = deg['usure_nouvelle'].apply(lambda x: math.ceil(x * 100) / 100)

# !!! Peut être temporaire !!!
# drop de l'avion B737_4325 qui est en double dans la table aeronef et donc génère des doublons dans les autres tables.
# la correction est possible mais on ne peut pas l'automatisé car il n'exitse pas de colonne dernier vol dans les df que l'on récupère
# Liste des DataFrames
list_of_dataframes = [aero, compo, deg, vol]

# Boucle pour itérer sur chaque DataFrame
for df in list_of_dataframes:
    
    indices_a_supprimer = df[df['ref_aero'] == 'B737_4325'].index
    
    df.drop(indices_a_supprimer, inplace=True)
# !!! Peut être temporaire !!!

In [4]:
compo = compo.drop(columns=['desc'],axis=1)
deg = deg[['ref_aero','ref_compo','usure_nouvelle','measure_day']]
vol = vol[['ref_aero','jour_vol','time_en_air','etat_voyant']]

In [5]:
# création du df maitre
maitre = aero.merge(compo, on='ref_aero', how ='left')

In [6]:
# travail sur le df degradation pour préparer la création des colonnes
pivot_deg = deg.pivot_table(
    values='usure_nouvelle',
    index=['ref_aero','ref_compo'],
    columns=['measure_day'],
    aggfunc='first'
).reset_index()

# fonction pour créer les colonnes de mesure/jour et classement
def remove_time_from_date(date_str):
    return re.sub(r'\s00:00:00$', '', str(date_str))

pivot_deg.columns = pivot_deg.columns[:2].tolist() + ['usure_' + remove_time_from_date(col) if col != 'mesure_day' else str(col) for col in pivot_deg.columns[2:]]


In [7]:
# Récupérer le nom des colonnes
colonnes = pivot_deg.columns

for t in range (33):
    # Parcourir les colonnes en commençant par la dernière jusqu'à la quatrième
    for i in range(len(colonnes) - 1, 3, -1):
        nom_colonne = colonnes[i]
        # Vérifier si la colonne contient des valeurs NaN
        if pivot_deg[nom_colonne].isnull().any():
            # Trouver la prochaine colonne non-NaN
            j = i - 1
            while j >= 0 and pivot_deg[colonnes[j]].isnull().all():
                j -= 1
            if j >= 0:
                # Remplacer les NaN par les valeurs de la prochaine colonne non-NaN
                next_non_nan_column = pivot_deg[colonnes[j]]
                pivot_deg.loc[pivot_deg[nom_colonne].isnull(), nom_colonne] = next_non_nan_column

In [8]:
for t in range (35):
# Parcourir les colonnes en commençant par la quatrième jusqu'à la dernière
    for i in range(2, len(colonnes)):
        nom_colonne = colonnes[i]
    # Vérifier si la colonne contient des valeurs NaN
        if pivot_deg[nom_colonne].isnull().any():
        # Trouver la prochaine colonne non-NaN
            j = i + 1
            while j < len(colonnes) and pivot_deg[colonnes[j]].isnull().all():
                j += 1
            if j < len(colonnes):
            # Remplacer les NaN par les valeurs de la prochaine colonne non-NaN
                next_non_nan_column = pivot_deg[colonnes[j]]
                pivot_deg.loc[pivot_deg[nom_colonne].isnull(), nom_colonne] = next_non_nan_column

In [9]:
# Sélectionner uniquement les colonnes de mesure, en excluant les colonnes 'ref_aero' et 'ref_compo'
mesure_columns = [col for col in pivot_deg.columns if col.startswith('usure')]

# Créer un DataFrame pour stocker les colonnes d'évolution
evolution_df = pd.DataFrame()


for i in range(1, len(mesure_columns)):
    # Nom de la nouvelle colonne d'évolution
    evolution_col_name = f"evolution_{mesure_columns[i]}"
    
    # Calculer la différence entre chaque paire de colonnes adjacentes, en ignorant les valeurs nulles
    evolution_df[evolution_col_name] = pivot_deg[mesure_columns[i]] - pivot_deg[mesure_columns[i-1]]

    # Remplacer les valeurs où la mesure précédente ou la mesure actuelle est nulle par NaN
    evolution_df.loc[pivot_deg[mesure_columns[i]] == 0, evolution_col_name] = pd.NA
    evolution_df.loc[pivot_deg[mesure_columns[i-1]] == 0, evolution_col_name] = pd.NA

# Concaténer le DataFrame d'évolution avec le DataFrame original
pivot_deg = pd.concat([pivot_deg, evolution_df], axis=1)


In [10]:
# Réorganiser les colonnes par date croissante
colonnes_triees = sorted(pivot_deg.columns[2:], key=lambda x: pd.to_datetime(x.split('_')[-1]))

# Créer une liste des colonnes dans l'ordre souhaité
colonnes_ordre = pivot_deg.columns[:2].tolist() + colonnes_triees

# Réorganiser les colonnes dans le DataFrame
pivot_deg = pivot_deg[colonnes_ordre]


In [11]:
pivot_deg

,ref_aero,ref_compo,usure_2024-01-04,usure_2024-01-05,evolution_usure_2024-01-05,usure_2024-01-06,evolution_usure_2024-01-06,usure_2024-01-07,evolution_usure_2024-01-07,usure_2024-01-08,...,usure_2024-02-01,evolution_usure_2024-02-01,usure_2024-02-02,evolution_usure_2024-02-02,usure_2024-02-03,evolution_usure_2024-02-03,usure_2024-02-04,evolution_usure_2024-02-04,usure_2024-02-05,evolution_usure_2024-02-05
0,A320_0691,AILA320-A320_0691-20,45.99,45.99,0.0,45.99,0.0,45.99,0.0,45.99,...,51.20,0.64,51.34,0.14,51.34,0.00,51.57,0.23,52.08,0.51
1,A320_0691,AUTA320-A320_0691-4,41.03,41.03,0.0,41.03,0.0,41.03,0.0,41.03,...,46.56,0.68,46.71,0.15,46.71,0.00,46.96,0.25,47.50,0.54
2,A320_0691,DETA320-A320_0691-37,5.47,5.47,0.0,5.47,0.0,5.47,0.0,5.47,...,12.18,0.82,12.36,0.18,12.36,0.00,12.66,0.30,13.32,0.66
3,A320_0691,ECLA320-A320_0691-30,76.27,76.27,0.0,76.27,0.0,76.27,0.0,76.27,...,1.45,0.64,1.60,0.15,1.60,0.00,1.83,0.23,2.35,0.52
4,A320_0691,ECLA320-A320_0691-38,35.17,35.17,0.0,35.17,0.0,35.17,0.0,35.17,...,40.86,0.70,41.02,0.16,41.02,0.00,41.27,0.25,41.83,0.56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10345,E175_6334,SYSE175-E175_6334-9,72.52,72.52,0.0,72.52,0.0,72.52,0.0,72.52,...,75.11,0.00,75.58,0.47,75.92,0.34,75.92,0.00,75.92,0.00
10346,E175_6334,TOIE175-E175_6334-35,51.36,51.36,0.0,51.36,0.0,51.36,0.0,51.36,...,54.63,0.00,55.22,0.59,55.64,0.42,55.64,0.00,55.64,0.00
10347,E175_6334,TRAE175-E175_6334-17,79.07,79.07,0.0,79.07,0.0,79.07,0.0,79.07,...,1.27,0.00,1.82,0.55,2.21,0.39,2.21,0.00,2.21,0.00
10348,E175_6334,TRAE175-E175_6334-18,41.57,41.57,0.0,41.57,0.0,41.57,0.0,41.57,...,44.62,0.00,45.17,0.55,45.56,0.39,45.56,0.00,45.56,0.00


In [12]:
# merge sur le maitre
maitre = maitre.merge(pivot_deg, on=['ref_aero', 'ref_compo'], how='left')

In [13]:
def create_pivot_column(df, variable):
    pivot_result = df.pivot_table(
        index='ref_aero',
        columns=['jour_vol'],
        values=[variable],
        aggfunc='sum',
        fill_value=0
    ).reset_index()

    pivot_result.columns = ['{}_{}'.format(variable, re.sub(r'^{}_'.format(variable), '', str(col[1])).split()[0]) if col[1] else col[0] for col in pivot_result.columns]
    pivot_result = pivot_result.rename(columns={'{}_NaT'.format(variable): 'ref_aero'})

    return pivot_result

# Appliquer la fonction à chaque variable
variables = ['etat_voyant','time_en_air']

for variable in variables:
    pivot_result = create_pivot_column(vol, variable)
    vol = pd.merge(vol, pivot_result, on='ref_aero', how='left')
    vol = vol.drop([variable], axis=1)

In [14]:
vol = vol.drop('jour_vol',axis=1)

In [15]:
# merge sur le maitre
maitre = maitre.merge(vol, on=['ref_aero'], how='left')

In [16]:
maitre = maitre.drop_duplicates(subset='ref_compo')

In [17]:
maitre = maitre.drop("taux_usure_actuel",axis=1)

In [18]:
maitre.columns

Index(['ref_aero', 'type_model', 'debut_service', 'last_maint',
       'en_maintenance', 'end_maint', 'ref_compo', 'categorie', 'lifespan',
       'cout',
       ...
       'time_en_air_2024-01-22', 'time_en_air_2024-01-23',
       'time_en_air_2024-01-29', 'time_en_air_2024-01-30',
       'time_en_air_2024-01-31', 'time_en_air_2024-02-01',
       'time_en_air_2024-02-02', 'time_en_air_2024-02-03',
       'time_en_air_2024-02-04', 'time_en_air_2024-02-05'],
      dtype='object', length=117)

In [19]:
def deplacer_colonne(df, nom_colonne, nouvelle_position):
    # Vérifiez si la colonne existe dans le DataFrame
    if nom_colonne in df.columns:
        # Retirez la colonne du DataFrame
        col = df[nom_colonne]
        df = df.drop(columns=[nom_colonne])

        # Insérez la colonne à la nouvelle position
        df.insert(nouvelle_position, nom_colonne, col)

        print("Colonne déplacée avec succès.")
    else:
        print("La colonne spécifiée n'existe pas dans le DataFrame.")

    return df

In [20]:
from datetime import datetime
# Calculer la date actuelle
date_actuelle = datetime.now()

# Calculer l'âge de l'avion en soustrayant la date de début de service de la date actuelle
maitre['age_avion'] = (date_actuelle - maitre['debut_service']).dt.days  # Âge en jours


In [21]:
deplacer_colonne(maitre, 'age_avion', 3)

Colonne déplacée avec succès.


,ref_aero,type_model,debut_service,age_avion,last_maint,en_maintenance,end_maint,ref_compo,categorie,lifespan,...,time_en_air_2024-01-22,time_en_air_2024-01-23,time_en_air_2024-01-29,time_en_air_2024-01-30,time_en_air_2024-01-31,time_en_air_2024-02-01,time_en_air_2024-02-02,time_en_air_2024-02-03,time_en_air_2024-02-04,time_en_air_2024-02-05
0,E175_4124,E175,2003-10-01,7434,2023-11-01,False,NaT,REAE175-E175_4124-0,Composants Critiques,11950,...,0.0,0.0,0.0,2.6,8.5,7.8,8.2,6.6,0.0,0.0
12,E175_4124,E175,2003-10-01,7434,2023-11-01,False,NaT,REAE175-E175_4124-1,Composants Critiques,14014,...,0.0,0.0,0.0,2.6,8.5,7.8,8.2,6.6,0.0,0.0
24,E175_4124,E175,2003-10-01,7434,2023-11-01,False,NaT,SYSE175-E175_4124-2,Composants Critiques,12519,...,0.0,0.0,0.0,2.6,8.5,7.8,8.2,6.6,0.0,0.0
36,E175_4124,E175,2003-10-01,7434,2023-11-01,False,NaT,ORDE175-E175_4124-3,Composants Critiques,11612,...,0.0,0.0,0.0,2.6,8.5,7.8,8.2,6.6,0.0,0.0
48,E175_4124,E175,2003-10-01,7434,2023-11-01,False,NaT,AUTE175-E175_4124-4,Composants Critiques,10655,...,0.0,0.0,0.0,2.6,8.5,7.8,8.2,6.6,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79840,B767_6363,B767,2016-07-06,2772,2023-10-07,False,NaT,SYSB767-B767_6363-40,Composants Secondaires,12602,...,0.0,5.3,0.0,0.0,0.0,0.0,0.0,2.3,0.9,8.5
79847,B767_6363,B767,2016-07-06,2772,2023-10-07,False,NaT,PORB767-B767_6363-41,Composants Secondaires,11227,...,0.0,5.3,0.0,0.0,0.0,0.0,0.0,2.3,0.9,8.5
79854,B767_6363,B767,2016-07-06,2772,2023-10-07,False,NaT,HUBB767-B767_6363-42,Composants Secondaires,14930,...,0.0,5.3,0.0,0.0,0.0,0.0,0.0,2.3,0.9,8.5
79861,B767_6363,B767,2016-07-06,2772,2023-10-07,False,NaT,ECLB767-B767_6363-43,Composants Secondaires,11837,...,0.0,5.3,0.0,0.0,0.0,0.0,0.0,2.3,0.9,8.5


In [22]:
for i in maitre.columns:
    print(i)

ref_aero
type_model
debut_service
last_maint
en_maintenance
end_maint
ref_compo
categorie
lifespan
cout
usure_2024-01-04
usure_2024-01-05
evolution_usure_2024-01-05
usure_2024-01-06
evolution_usure_2024-01-06
usure_2024-01-07
evolution_usure_2024-01-07
usure_2024-01-08
evolution_usure_2024-01-08
usure_2024-01-09
evolution_usure_2024-01-09
usure_2024-01-10
evolution_usure_2024-01-10
usure_2024-01-11
evolution_usure_2024-01-11
usure_2024-01-12
evolution_usure_2024-01-12
usure_2024-01-13
evolution_usure_2024-01-13
usure_2024-01-14
evolution_usure_2024-01-14
usure_2024-01-15
evolution_usure_2024-01-15
usure_2024-01-16
evolution_usure_2024-01-16
usure_2024-01-17
evolution_usure_2024-01-17
usure_2024-01-18
evolution_usure_2024-01-18
usure_2024-01-19
evolution_usure_2024-01-19
usure_2024-01-20
evolution_usure_2024-01-20
usure_2024-01-21
evolution_usure_2024-01-21
usure_2024-01-22
evolution_usure_2024-01-22
usure_2024-01-23
evolution_usure_2024-01-23
usure_2024-01-24
evolution_usure_2024-01-24

In [23]:
# Liste des colonnes de time_en_air_ à parcourir
colonnes_time_en_air = [colonne for colonne in maitre.columns if colonne.startswith("time_en_air_")]

# Parcours des colonnes de time_en_air_
for colonne_time_en_air in colonnes_time_en_air:
    # Extraire la date de la colonne time_en_air_
    date = colonne_time_en_air.split("_")[-1]
    
    # Vérifier si la valeur de time_en_air_ pour cette date est > 0
    if maitre[colonne_time_en_air].iloc[0] > 0:
        # Calculer le taux d'usure
        colonne_evolution_usure = f"evolution_usure_{date}"
        taux_usure = maitre[colonne_evolution_usure] / maitre[colonne_time_en_air]
        
        # Créer une nouvelle colonne taux_usure_
        colonne_taux_usure = f"taux_usure_{date}"
        maitre[colonne_taux_usure] = taux_usure


In [24]:
# travail sur le maitre pour organiser les colonnes
columns = maitre.columns

# Extraire les dates de chaque colonne et créer une liste de tuples (date, colonne)
date_column_tuples = [(re.search(r'\d{4}-\d{2}-\d{2}', col).group(), col) for col in columns[8:] if re.search(r'\d{4}-\d{2}-\d{2}', col)]

# Trier la liste de tuples par date
sorted_date_column_tuples = sorted(date_column_tuples, key=lambda x: x[0])

# Extraire la liste triée des colonnes
sorted_columns = [col[1] for col in sorted_date_column_tuples]

# Réorganiser les colonnes selon la spécification demandée
desired_order = ['type_model', 'ref_aero', 'debut_service','age_avion', 'last_maint', 'end_maint', 'ref_compo', 'categorie','cout', 'lifespan']
sorted_columns = desired_order + sorted_columns

# Créer un DataFrame avec les colonnes triées
maitre = maitre[sorted_columns]